# LLM 연동 연습문제

## 문제 1-1 : 기본 Chain 만들기 - AI 요리사
### 문제 설명
#### 사용자가 재료를 입력하면 그 재료로 만들 수 있는 요리를 추천해주는 간단한 AI 요리사를 만들어보세요.
#### 요구사항
#### PromptTemplate을 사용하여 프롬프트 작성
#### 사용자가 입력한 재료를 받아서 만들 수 있는 요리 추천
#### ChatOpenAI 모델 사용
#### StrOutputParser로 결과를 문자열로 출력
#### LCEL(|) 문법을 사용하여 체인 연결
### 구현 조건
#### 입력: 재료명 (예: "토마토, 양파, 치즈")
#### 출력: 추천 요리와 간단한 레시피
### 예상 실행 결과
#### 입력: "계란, 밥, 김치"
#### 출력: 
#### 계란과 밥, 김치로 만들 수 있는 요리를 추천드립니다!

#### 추천 요리: 김치볶음밥
#### 재료: 계란, 밥, 김치
#### 조리법:
#### 1. 팬에 기름을 두르고 김치를 볶아주세요
#### 2. 밥을 넣고 함께 볶아주세요
#### 3. 계란을 풀어서 넣고 잘 섞어주세요
#### ...


In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

prompt_template = PromptTemplate.from_template(
    """당신은 '재료 기반 요리 추천' AI 요리사입니다.
아래 '입력 재료'로 만들 수 있는 한 가지 한국 가정식 요리를 추천하고,
간단한 레시피를 단계별로 알려주세요.

요건:
- 추천 요리 이름 1개
- 필요한 재료(입력 재료 위주, 추가가 필요하면 최소한으로)
- 조리법: 1, 2, 3... 단계별
- 말투는 친절하고 간결하게

입력 재료: {ingredients}

출력 형식 예:
계란과 밥, 김치로 만들 수 있는 요리를 추천드립니다!

추천 요리: 김치볶음밥
재료: 계란, 밥, 김치
조리법:
1. ...
2. ...
3. ...
"""
)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="openai/gpt-oss-120b",  # Spring AI와 동일한 모델
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"ingredients": "토마토, 양파, 치즈"})
pprint(response)

('토마토, 양파, 치즈로 만들 수 있는 요리를 추천드립니다!\n'
 '\n'
 '**추천 요리:** 토마토 치즈 계란찜  \n'
 '\n'
 '**재료:** 토마토, 양파, 치즈, 계란\u202f2개, 물\u202f½컵, 소금·후추 약간 (선택 사항: 파)\n'
 '\n'
 '**조리법:**  \n'
 '1. 토마토와 양파를 작게 깍둑썰기합니다.  \n'
 '2. 작은 팬에 기름을 살짝 두르고 양파를 투명해질 때까지 볶은 뒤, 토마토를 넣고 1분 정도 살짝 익혀줍니다.  \n'
 '3. 볼에 계란을 풀고 물, 소금·후추를 넣어 섞은 뒤, 2번 재료 위에 붓습니다.  \n'
 '4. 그 위에 갈은 치즈(또는 얇게 썬 치즈)를 골고루 뿌리고, 뚜껑을 덮어 약불에서 5~7분 정도 찝니다. (계란이 살짝 부드럽게 '
 '익으면 완성)  \n'
 '5. 원한다면 다진 파를 살짝 뿌려 마무리합니다.  \n'
 '\n'
 '간단하지만 부드러운 토마토와 치즈 풍미가 살아 있는 한 끼가 완성됐어요! 맛있게 드세요.')


## 문제 1-2 : 2단계 Multi Chain 만들기 - 여행지 정보 시스템
### 문제 설명
#### 사용자가 특정 도시나 국가를 입력하면, 해당 지역의 대표적인 관광 명소를 추천하고 그 명소에 대한 상세 정보(역사, 특징, 방문 팁 등)를 알려주는 2단계 체인을 구현해 보세요.
### 요구사항
#### 1단계 체인: 여행지(도시/국가)를 입력받아 대표 명소 1가지 추천
#### 2단계 체인: 추천받은 명소의 상세 정보 제공
#### ChatPromptTemplate 사용 
#### : “system”  과 “user” 메시지를 지정합니다.
#### 두 체인을 LCEL로 연결
#### 각 단계의 결과를 모두 출력하여 과정 확인
### 구현 조건
#### 입력: 여행지 (예: "로마", "뉴욕", "도쿄")
#### 1단계 출력: 추천 명소 이름
#### 2단계 출력: 명소에 대한 상세 정보 (역사, 특징, 방문 팁 등)
#### 예상 실행 결과
#### 입력: "로마" 1단계 결과: "콜로세움" 
#### 추천 2단계 결과: 명소: 콜로세움 
### 역사:
#### 로마 시대에 건설된 거대한 원형 경기장입니다.
#### ... (이하 역사 설명)
#### 특징:
#### 검투사들의 시합이 열리던 장소입니다.
#### ... (이하 특징 설명)
#### 방문 팁:
#### 미리 온라인으로 표를 예매하면 좋습니다.
#### ... (이하 방문 팁)

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="openai/gpt-oss-120b",
    temperature=0.7
)

prompt_step1 = ChatPromptTemplate.from_messages([
    ("system", "너는 여행 추천 도우미야. 사용자가 입력한 도시/국가에서 대표 관광 명소 하나만 추천해줘. 출력은 명소 '이름'만 한 줄로."),
    ("user",   "여행지: {location}\n조건: 대표 명소 1개만, 다른 설명은 쓰지 말 것.")
])

chain_step1 = prompt_step1 | llm | StrOutputParser()

prompt_step2 = ChatPromptTemplate.from_messages([
    ("system", "너는 여행 가이드야. 주어진 명소에 대해 간결하지만 알찬 정보를 제공해줘."),
    ("user",
     "여행지: {location}\n명소: {place}\n\n"
     "아래 형식으로 알려줘.\n"
     "명소: {place}\n"
     "역사:\n- …\n- …\n"
     "특징:\n- …\n- …\n"
     "방문 팁:\n- …\n- …")
])

chain_step2 = prompt_step2 | llm | StrOutputParser()

location = ["로마", "뉴욕", "도쿄"]
step1_output = chain_step1.invoke({"location": location}).strip()
print("입력 :", location)
print("1단계 결과:", step1_output, "\n")

step2_output = chain_step2.invoke({"location": location, "place": step1_output})
print("추천 2단계 결과:\n")
print(step2_output)

입력 : ['로마', '뉴욕', '도쿄']
1단계 결과: 콜로세움
자유의 여신상
도쿄 스카이트리 

추천 2단계 결과:

**명소: 콜로세움**  
역사:  
- 서기 72년 베스파시아누스 황제가 건설을 시작해 80년에 완공된 고대 로마의 원형극장.  
- 검투사 경기, 사냥, 공개 처형 등 다양한 공개 오락이 열리던 장소였으며, 로마 제국의 권위와 엔터테인먼트를 상징.  
- 5세기 이후 지진과 약탈으로 파손됐지만, 중세에는 요새와 교회로 재활용되었다가 18세기에 복원 작업이 시작돼 현재의 모습이 되었다.  

특징:  
- 직경 약 189 m, 높이 48 m에 달하는 거대한 구조물로, 5만~8만 명을 수용할 수 있었다.  
- 아치형 구조와 복합적인 지하 설계(동물 사육장, 전투용 장비 보관소 등)로 건축 기술의 정점을 보여준다.  
- 야간 조명과 내부 투어(지하, 상층 관람석)로 과거와 현재를 동시에 체험할 수 있다.  

방문 팁:  
- **조기 입장**: 아침 8시~9시 사이에 입장하면 인파를 피하고 시원한 분위기에서 관람 가능.  
- **온라인 티켓**: 사전 예약 시 대기 시간을 크게 줄일 수 있으며, “입장 + 지하 투어” 패키지를 추천.  
- **편한 신발**: 바닥이 고르지 않아 오래 걸어야 하니 편한 운동화 착용을 권장.  

---

**명소: 자유의 여신상**  
역사:  
- 프랑스가 미국 독립 100주년을 기념해 선물한 동상으로, 1886년 뉴욕 항에 설치.  
- 프랑스 조각가 프레데리크·오귀스트 바르톨디가 디자인하고, 에펠탑의 엔지니어 구스타브·에펠이 내부 구조를 설계했다.  
- 1916년 미국 입법부가 “자유의 여신상”을 국가 기념물로 지정했으며, 1984년 유네스코 세계문화유산에 등재.  

특징:  
- 높이 46 m(기초 포함 93 m)이며, 구리 판을 겹쳐 만든 외피가 자연산화로 녹색을 띤다.  
- 오른손에 횃불, 왼손에 독립 선언서 복제본을 들고 있어 ‘자유와 민주주의’를 상징.  
- 내부에는 엘리베이터와 

## 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용 
### 뉴스 키워드 추출기
### 1) 문제 설명
#### FewShotPromptTemplate을 사용하여 뉴스 기사에서 핵심 키워드 3개를 추출하는 시스템을 구현해보세요. 주어진 예시들을 참고하여 일관된 형식으로 키워드를 추출해야 합니다.
### 2) 요구사항
#### FewShotPromptTemplate 사용
#### 최소 3개의 예시(examples) 포함
#### 뉴스 텍스트에서 핵심 키워드 3개 추출
#### 일관된 출력 형식 유지
#### 다양한 분야의 뉴스로 테스트
### 3) 예시 데이터
#### examples = [
#### {
#### "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
#### "keywords": "삼성전자, 인공지능, 엔비디아"
#### },
#### {
#### "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
####  "keywords": "세계보건기구, 건강위기, 국제협력"
#### },
#### 추가 예시 필요
#### ]
#### 테스트 뉴스
#### "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."
#### 예상 출력 형식
#### 키워드: 제미나이, 구글AI, 개발자
### 4) 구현 힌트
#### from langchain_core.prompts import (
#### FewShotChatMessagePromptTemplate, 
#### ChatPromptTemplate
#### )

### 1. 예시 프롬프트 (대화형)
#### example_prompt = ChatPromptTemplate.from_messages([
#### ("human", "{news}"),
#### ("ai", "키워드: {keywords}")
#### ])
#### examples = [
#### {
#### "news": "삼성전자가 AI 가속기를 출시할 예정이다.",
#### "keywords": "삼성전자, AI가속기, 출시"
#### },
#### {
#### "news": "WHO가 국제 협력의 중요성을 강조했다.",
#### "keywords": "WHO, 국제협력, 보건"
#### }
#### ]

### 2. Few-Shot 프롬프트
#### few_shot_prompt = FewShotChatMessagePromptTemplate(
#### example_prompt=example_prompt,
#### examples=examples
#### )

### 3. 최종 프롬프트
#### final_prompt = ChatPromptTemplate.from_messages([
#### ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
#### few_shot_prompt,
#### ("human", "{input}")
#### ])

In [11]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="openai/gpt-oss-120b",
    temperature=0.2,
)

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

examples = [
    {
        "news": "삼성전자가 내년 초 자체 개발한 인공지능 가속기를 출시할 예정이다. 이는 엔비디아의 독점에 도전하려는 시도로 해석된다.",
        "keywords": "삼성전자, 인공지능가속기, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 새로운 전염병 확산에 대비해 국제 협력 강화를 요청했다.",
        "keywords": "세계보건기구, 전염병, 국제협력"
    },
    {
        "news": "테슬라는 차세대 전기차 플랫폼 개발을 가속화하며 생산 단가 절감을 목표로 하고 있다.",
        "keywords": "테슬라, 전기차플랫폼, 생산단가"
    },
]

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 뉴스 키워드 추출 전문가다. 아래 규칙을 반드시 지켜라.\n"
     "1) 핵심 키워드 정확히 3개\n"
     "2) 형식: '키워드: A, B, C'\n"
     "3) 불필요한 설명/문장 금지, 키워드만 출력\n"
     "4) 고유명사/핵심개념 위주로 간결하게"),
    few_shot_prompt,
    ("human", "{input}")
])

chain = final_prompt | llm | StrOutputParser()

test_news = (
    "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 "
    "제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 "
    "사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. "
    "내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."
)

print(chain.invoke({"input": test_news}))

more_tests = [
    "한국은행이 기준금리를 동결했다. 물가 안정과 경기 둔화를 종합적으로 고려한 결정으로 풀이된다.",
    "프리미어리그에서 손흥민이 멀티골을 기록하며 팀 승리에 크게 기여했다.",
    "유럽연합(EU)은 데이터 보호 규정을 강화해 글로벌 IT 기업들의 준수를 요구했다."
]
for t in more_tests:
    print(chain.invoke({"input": t}))


키워드: 제미나이 2.0, Google AI Studio, Vertex AI
키워드: 한국은행, 기준금리, 물가안정
키워드: 프리미어리그, 손흥민, 멀티골
키워드: 유럽연합, 데이터보호규정, 글로벌IT기업
